In [4]:
from Project3.fluid import Fluid

# inputsArr is a 2D array of ordered pairs (m_C,p2) where m_c is equal to the continuous mass flow and p2 is the pump discharge pressure



# Initialize an empty array to start
inputsArr = []

# Define the values that will be at the top left of the array
start_m_c = 0.1
start_p2 = 16

# Define the increments
m_c_increment = 0.05
p2_increment = 0.1

# Define the number of rows and columns
num_rows = 100
num_cols = 100

# Loop through rows
for i in range(num_rows):
    row = []
    m_c = start_m_c + i * m_c_increment  # Increment m_c for each row
    p2 = start_p2  # Reset p2 for each row
    # Loop through columns
    for j in range(num_cols):
        row.append((m_c, p2))  # Append the ordered pair to the row
        p2 += p2_increment  # Increment p2 for each column
    inputsArr.append(row)  # Append the row to the 2D array


#Givens
m_dot_s = 3.2 # mass flow of the continuous supply being sequestered (s for sequestration)
#State 5 is the incoming sequestration state 
T5 = 239.15 
P5 = 6.0 

#For each pair of free variables, we will compute the net power produced
net_pow_arr = []
for i in range(num_rows):
    pow_row = []
    for j in range(num_cols):
        #free variables
        m_dot_c = inputsArr(i,j)[0]
        P_pump_discharge =inputsArr(i,j)[1]

        #given
        m_dot_s = 3.2

        #State 1
        m_dot_1 = m_dot_s + m_dot_c
        T1 = T5 # to match sequestration state
        P1 = P5 #to match sequestration state
        

        co2 = Fluid('co2')
        s1 = co2.entropy(T=T1, P=P1)
        h1 = co2.enthalpy(T = T1, P = P1)


        #State 2
        m_dot_2 = m_dot_s + m_dot_c
        s2 = s1 #Pump modeled as isentropic
        P2 = P_pump_discharge #Free variable

        h2 = co2.enthalpy(s = s2, P = P2)
        T2 = co2.temperature(s = s2, P = P2)

        pow_in = m_dot_2 * (h2 - h1)

        #State 3
        dP=0 #for now
        dT=0 #for now
        m_dot_3 = m_dot_c
        P3 = P2 - dP #dP is the pressure loss of the underground system
        T3 = T2 + dT #dT is the temperature increase after the underground system

        s3 = co2.entropy(T=T3, P=P3)
        h3 = co2.enthalpy(T = T3, P = P3)

        #State 4
        m_dot_4 = m_dot_c
        s4 = s3 #Turbine modeled as isentropic
        P4 = P1 # Condenser modeled as isobarric

        h4 = co2.enthalpy(s = s4, P = P4)
        pow_out = m_dot_4 * (h3 - h4)


        net_pow = pow_out-pow_in
        pow_row.append(net_pow)
    net_pow_arr.append(pow_row)


#find index of best net power
#start by guessing best index is (1,1)
i_best = 1
j_best = 1

#Get continuous mass flow and pump discharge pressure of optimized net power
m_c_best = net_pow_arr(i_best,j_best)[0]
P2_best = net_pow_arr(i_best,j_best)[1]


TypeError: 'list' object is not callable

In [5]:
#calculating specific speed for sizing pump and turbine

omega_t #rotational speed, adjust to get a reasonable efficiency
Q_t = m_dot_c / co2.density(T = T3, P = P3) #volumetric flow rate
enthalpy_change_t = h3 - h4

specific_speed_d_t = omega_t * Q_t^2 / enthalpy_change_t

omega_p #rotational speed, adjust to get a reasonable efficiency
Q_p = (m_dot_c + m_dot_s) / co2.density(T = T1, P = P1)#volumetric flow rate
enthalpy_change_p = h2 - h1

specific_speed_d_p = omega_p * Q_p^2 / enthalpy_change_p

NameError: name 'omega_t' is not defined